# Model training
This notebook experiments with different training methods and model architectures.

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.layers import Conv2D, Dropout, MaxPooling2D, Dense
from keras.applications.vgg19 import VGG19, decode_predictions

Using TensorFlow backend.


In [3]:
SUB_TASK_B_PATH = "../data/Task2_offlineRec/subTask_symbols/"
TRAIN_IMG_PATH = SUB_TASK_B_PATH + "Train/"

# 2014 set of training data
train_2014 = TRAIN_IMG_PATH + "data_png_trainingSymbols/"
class_2014 = TRAIN_IMG_PATH + "2014_symbols_GT.txt"

In [4]:
IMG_WIDTH = IMG_HEIGHT = 224 # this is the default for vgg19
IMG_CHANNELS = 3
MODEL_DIR = ""
TRAIN_DIR = ""
VALID_DIR = ""

In [5]:
def init_model():
    """Loads vgg19 and adds layers ontop
    
    Returns:
        The Keras model
    """
    vgg19 = VGG19(include_top=False,
                       weights='imagenet',
                       input_shape=(IMG_X, IMG_Y, IMG_CHANNELS),
                       pooling=None)
    
    # create a Sequential model from VGG19 layers & weights
    base_model = Sequential()
    for l in vgg19.layers:
        base_model.add(l)
    
    top_model = keras.Sequential()
    # convolution
    top_model.add(Conv2D(32, kernel_size=3, input_shape=(100, 100, 3), activation='relu'))
    top_model.add(MaxPooling2D(pool_size=2))
    top_model.add(Dropout(0.1))
    
    top_model.add(Conv2D(32, kernel_size=3, activation='relu'))
    top_model.add(MaxPooling2D(pool_size=2))
    top_model.add(Dropout(0.1))
    
    # classification
    top_model.add(Flatten())
    top_model.add(Dense(32, activation='relu'))
    top_model.add(Dense(100, activation='softmax'))
    
    base_model.add(vgg19)
    
    # lock top layers for model
    for layer in base_model.layers:
        layer.trainable = False
    
    return base_model

In [6]:
def train_model(model, images, labels):
    """Compiles and trains the model given the dataset
    
    The model is also saved after it is trained
    
    Args:
        images:
        labels:
        
    Returns:
        The trained model
    """
    train_samples = 1000
    validation_samples = 1000
    epochs = 30
    batch_size = 16
    
    # For a multi-class classification problem
    model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    one_hot_labels = keras.utils.to_categorical(labels, num_classes=NUM_CLASSES)

    model.fit(images, labels, epochs=10, batch_size=32)

In [7]:
def save_model(model, path):
    """Saves a model to location specified in path
    
    Args:
        path: The path to save the trained model
    """
    model.save("model.h5")

In [8]:
def load_model(path):
    """Returns a pretrained model
    
    Args:
        path: The path to the trained model
        
    Returns:
        The pretrained model with architecture and weights
    """
    return load_model(path)

In [9]:
# main
model = VGG19()
img = image.img_to_array(image.load_img(train_2014 + "iso7177.png", target_size=(224, 224)))
# img = VGG19.preprocess_input(img)
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
x = model.predict(img)
label = decode_predictions(x)
label = label[0][0]
print('%s (%.2f%%)' % (label[1], label[2]*100))
# train_model(model, images, labels)
# save_model(model, MODEL_PATH)

container_ship (15.95%)
